In [1]:
import cv2 as cv
import math
import numpy as np
import os
import random

class Point:
    x = 0.0
    y = 0.0

In [2]:
# Constants
image_w = 416
image_h = 416

number_of_images = 20000

max_triangles = 0
max_pentagons = 0
max_rectangles = 0
max_stars = 0
max_randoms = 4

max_size_rectangle = 150
max_size_pentagon  = 150
max_size_triangle  = 150
max_size_star      = 150
max_size_random    = 150

skew_rectangle  = True
skewx_rectangle = 40

min_size_rectangle = 80
min_size_pentagon  = 80
min_size_triangle  = 80
min_size_star      = 80
min_size_random    = 80

# Star setup
number_peaks = 5

min_angle = -1
max_angle = 1

max_attempts = 100

occlusion_allowed = False
rotation_allowed = False

img_folder_path = os.path.join('/','mnt','F0CC2854CC28177E','datasets','convex_polygon')
txt_output = '_desc.txt'
image_folder = os.path.join(img_folder_path, 'images')
bg_folder = os.path.join('/','mnt','F0CC2854CC28177E','datasets','backgrounds')
bg_num = 231

In [ ]:
# Delete fileto save output if exists or create a new one
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

text_file = open(image_folder + '/' + txt_output, "w")
text_file.close()

# Generate image and save it
for i in range(number_of_images):
    img = np.zeros((image_w,image_h,3), np.uint8)
    img_bg = np.zeros((image_w,image_h,3), np.uint8)
    failed = ''
    txt_line = ''
    
    if (bg_folder != ''):
        bg = cv.imread(bg_folder + '/' + str(random.randint(1,bg_num)) + '.jpg')
        #bg = cv.cvtColor(bg, cv.COLOR_BGR2RGB)
        posx = random.randint(0,bg.shape[1]-image_w-1)
        posy = random.randint(0,bg.shape[0]-image_h-1)
        img_bg  = bg[posy:posy+image_h,posx:posx+image_w,...]
        #print('save')
        #file_name = 'img_x' + str(i) + '.png'
        #file_path = image_folder + '/' + file_name        
        #cv.imwrite(file_path, img)    
        

    # Rectangles - class 0
    rectangles = random.randint(0,max_rectangles)
    
    for r in range(rectangles):
        fine = False
        size = 0
        attempts = 0
        angle = 0
        
        center_x = 0
        center_y = 0
        
        pt1 = (0,0)
        pt2 = (0,0)
        pt3 = (0,0)        
        pt4 = (0,0)         

        rectangle_cnt = []
              
        while attempts < max_attempts:
            size = random.randint(min_size_rectangle,max_size_rectangle)
            angle = random.randint(min_angle,max_angle)              
            
            center_x = random.randint(size+5,image_w-size-1)
            center_y = random.randint(size+5,image_h-size-1)
            
            left_up_x = center_x-int(size/2)
            left_up_y = center_y-int(size/2)
            
            pt1 = (left_up_x,left_up_y)
            pt2 = (left_up_x+size,left_up_y)
            pt3 = (left_up_x+size, left_up_y+size)
            pt4 = (left_up_x,left_up_y+size)
            
            if rotation_allowed == True:
                pt1 = ( \
                      (pt1[0] - center_x) * math.cos(angle) - (pt1[1] - center_y) * math.sin(angle), \
                      (pt1[1] - center_y) * math.cos(angle) + (pt1[0] - center_x) * math.sin(angle) \
                      )
                pt1 = (pt1[0]+center_x,pt1[1]+center_y)
                pt2 = ( \
                      (pt2[0] - center_x) * math.cos(angle) - (pt2[1] - center_y) * math.sin(angle), \
                      (pt2[1] - center_y) * math.cos(angle) + (pt2[0] - center_x) * math.sin(angle) \
                      )
                pt2 = (pt2[0]+center_x,pt2[1]+center_y)
                pt3 = ( \
                      (pt3[0] - center_x) * math.cos(angle) - (pt3[1] - center_y) * math.sin(angle), \
                      (pt3[1] - center_y) * math.cos(angle) + (pt3[0] - center_x) * math.sin(angle) \
                      )
                pt3 = (pt3[0]+center_x,pt3[1]+center_y)
                pt4 = ( \
                      (pt4[0] - center_x) * math.cos(angle) - (pt4[1] - center_y) * math.sin(angle), \
                      (pt4[1] - center_y) * math.cos(angle) + (pt4[0] - center_x) * math.sin(angle) \
                      )
                pt4 = (pt4[0]+center_x,pt4[1]+center_y)        
                
            
            if (skew_rectangle):
                skewx = random.randint(-skewx_rectangle, skewx_rectangle)
                skewy = random.randint(-size, size//2)
                pt1 = (pt1[0]+skewx, pt1[1]+skewy)
                pt2 = (pt2[0]+skewx, pt2[1]+skewy)
            
            rectangle_cnt = np.array( [pt1, pt2, pt3, pt4] )
            
                        
            if occlusion_allowed == True:
                break;
                                
            img_shape = np.zeros((image_w,image_h,1), np.uint8)
            cv.drawContours(img_shape, [rectangle_cnt.astype(int)], 0, 1, cv.FILLED)
            
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            ret,img_gray = cv.threshold(img[:,:,0],0,1,cv.THRESH_BINARY)
            
            occlusion_test = cv.bitwise_and(img_shape,img_gray)
            
            white_pixels = cv.countNonZero(occlusion_test)
            
            attempts = attempts+1
            
            if (white_pixels > 0):
                continue;
            else:
                break;
                
        if (attempts == max_attempts):
            failed = 'FAILED_'
        
        r = random.randint(50,200)
        g = random.randint(50,200)
        b = random.randint(50,200)
            
        cv.drawContours(img, [rectangle_cnt.astype(int)], 0, (r,g,b), cv.FILLED)
        cv.drawContours(img_bg, [rectangle_cnt.astype(int)], 0, (r,g,b), cv.FILLED)
        
        # Bounding rect
        x,y,w,h = cv.boundingRect(rectangle_cnt.astype(int))
        
        # Add line to a file content        
        txt_line = txt_line + \
                   str(x) + ',' + str(y) + ',' + \
                   str(x+w-1) + ',' + str(y+h-1) + \
                   ',0,' + \
                   str(int(pt1[0])) + ',' + str(int(pt1[1])) + ',' + \
                   str(int(pt2[0])) + ',' + str(int(pt2[1])) + ',' + \
                   str(int(pt3[0])) + ',' + str(int(pt3[1])) + ',' + \
                   str(int(pt4[0])) + ',' + str(int(pt4[1])) + ' '

    # Pentagons - class 1
    pentagons = random.randint(0,max_pentagons)
    
    for c in range(pentagons):
        center_x = 0
        center_y = 0
        radius = 0
        attempts = 0
        angle = 0
        
        pt1 = (0,0)
        pt2 = (0,0)
        pt3 = (0,0)        
        pt4 = (0,0)        
        pt5 = (0,0)
        
        pentagon_cnt = []
        
        while attempts < max_attempts:
            size = random.randint(min_size_pentagon,max_size_pentagon)
            angle = random.randint(min_angle,max_angle)            
            
            center_x = random.randint(size+1,image_w-size-1)
            center_y = random.randint(size+1,image_h-size-1)
            
            radius=int(size/2)
            slope = int(radius*0.8)                        
            
            pt1 = (center_x+radius,center_y)
            pt2 = (center_x+slope,center_y+radius)
            pt3 = (center_x-slope, center_y+radius)
            pt4 = (center_x-radius,center_y)
            pt5 = (center_x,center_y-radius)
            
            if rotation_allowed == True:
                pt1 = ( \
                      (pt1[0] - center_x) * math.cos(angle) - (pt1[1] - center_y) * math.sin(angle), \
                      (pt1[1] - center_y) * math.cos(angle) + (pt1[0] - center_x) * math.sin(angle) \
                      )
                pt1 = (pt1[0]+center_x,pt1[1]+center_y)
                pt2 = ( \
                      (pt2[0] - center_x) * math.cos(angle) - (pt2[1] - center_y) * math.sin(angle), \
                      (pt2[1] - center_y) * math.cos(angle) + (pt2[0] - center_x) * math.sin(angle) \
                      )
                pt2 = (pt2[0]+center_x,pt2[1]+center_y)
                pt3 = ( \
                      (pt3[0] - center_x) * math.cos(angle) - (pt3[1] - center_y) * math.sin(angle), \
                      (pt3[1] - center_y) * math.cos(angle) + (pt3[0] - center_x) * math.sin(angle) \
                      )
                pt3 = (pt3[0]+center_x,pt3[1]+center_y)
                pt4 = ( \
                      (pt4[0] - center_x) * math.cos(angle) - (pt4[1] - center_y) * math.sin(angle), \
                      (pt4[1] - center_y) * math.cos(angle) + (pt4[0] - center_x) * math.sin(angle) \
                      )
                pt4 = (pt4[0]+center_x,pt4[1]+center_y)                
                pt5 = ( \
                      (pt5[0] - center_x) * math.cos(angle) - (pt5[1] - center_y) * math.sin(angle), \
                      (pt5[1] - center_y) * math.cos(angle) + (pt5[0] - center_x) * math.sin(angle) \
                      )
                pt5 = (pt5[0]+center_x,pt5[1]+center_y)                
                        
            pentagon_cnt = np.array( [pt1, pt2, pt3, pt4, pt5] )            
        
            if occlusion_allowed == True:
                break;
                            
            img_shape = np.zeros((image_w,image_h,1), np.uint8)
            cv.drawContours(img_shape, [pentagon_cnt.astype(int)], 0, 1, cv.FILLED)

            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            ret,img_gray = cv.threshold(img[:,:,0],0,1,cv.THRESH_BINARY)
            
            occlusion_test = cv.bitwise_and(img_shape,img_gray)
            
            white_pixels = cv.countNonZero(occlusion_test)
            
            attempts = attempts+1
            
            if (white_pixels > 0):
                continue;
            else:
                break;
                
        if (attempts == max_attempts):
            failed = 'FAILED_'
        
        r = random.randint(50,200)
        g = random.randint(50,200)
        b = random.randint(50,200) 

        cv.drawContours(img, [pentagon_cnt.astype(int)], 0, (r,g,b), cv.FILLED)
        cv.drawContours(img_bg, [pentagon_cnt.astype(int)], 0, (r,g,b), cv.FILLED)
        
        # Bounding rect
        x,y,w,h = cv.boundingRect(pentagon_cnt.astype(int))
        
        # Add line to a file content
        txt_line = txt_line + \
                   str(x) + ',' + str(y) + ',' + \
                   str(x+w-1) + ',' + str(y+h-1) + \
                   ',1,' + \
                   str(int(pt1[0])) + ',' + str(int(pt1[1])) + ',' + \
                   str(int(pt2[0])) + ',' + str(int(pt2[1])) + ',' + \
                   str(int(pt3[0])) + ',' + str(int(pt3[1])) + ',' + \
                   str(int(pt4[0])) + ',' + str(int(pt4[1])) + ',' + \
                   str(int(pt5[0])) + ',' + str(int(pt5[1])) + ' '
        
    # Triangles - class 2
    triangles = random.randint(0,max_triangles)
    
    for t in range(triangles):
        fine = False

        left_down_x = 0
        left_down_y = 0
        size = 0
        attempts = 0
        angle = 0
        
        center_x = 0
        center_y = 0
        
        pt1 = (0,0)
        pt2 = (0,0)
        pt3 = (0,0)
        
        triangle_cnt = []
        
        while attempts < max_attempts:
            size = random.randint(min_size_triangle,max_size_triangle)
            angle = random.randint(min_angle,max_angle)
            center_x = random.randint(size+1,image_w-size-1)
            center_y = random.randint(size+1,image_h-size-1)
            
            left_down_x = center_x-int(size/2)
            left_down_y = center_y+int(size/2)
           
            pt1 = (left_down_x,left_down_y)
            pt2 = (left_down_x+int(size/2),left_down_y-size)
            pt3 = (left_down_x+size, left_down_y)

            if rotation_allowed == True:
                pt1 = ( \
                      (pt1[0] - center_x) * math.cos(angle) - (pt1[1] - center_y) * math.sin(angle), \
                      (pt1[1] - center_y) * math.cos(angle) + (pt1[0] - center_x) * math.sin(angle) \
                      )
                pt1 = (pt1[0]+center_x,pt1[1]+center_y)
                pt2 = ( \
                      (pt2[0] - center_x) * math.cos(angle) - (pt2[1] - center_y) * math.sin(angle), \
                      (pt2[1] - center_y) * math.cos(angle) + (pt2[0] - center_x) * math.sin(angle) \
                      )
                pt2 = (pt2[0]+center_x,pt2[1]+center_y)
                pt3 = ( \
                      (pt3[0] - center_x) * math.cos(angle) - (pt3[1] - center_y) * math.sin(angle), \
                      (pt3[1] - center_y) * math.cos(angle) + (pt3[0] - center_x) * math.sin(angle) \
                      )
                pt3 = (pt3[0]+center_x,pt3[1]+center_y)
        
            triangle_cnt = np.array( [pt1, pt2, pt3] )
        
            if occlusion_allowed == True:
                break;
            
            img_shape = np.zeros((image_w,image_h,1), np.uint8)            
            cv.drawContours(img_shape, [triangle_cnt.astype(int)], 0, 1, cv.FILLED)

            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            ret,img_gray = cv.threshold(img[:,:,0],0,1,cv.THRESH_BINARY)
            
            occlusion_test = cv.bitwise_and(img_shape,img_gray)
            
            white_pixels = cv.countNonZero(occlusion_test)
            
            attempts = attempts+1
            
            if (white_pixels > 0):
                continue;
            else:
                break;    
                
        if (attempts == max_attempts):
            failed = 'FAILED_'
        
        r = random.randint(50,200)
        g = random.randint(50,200)
        b = random.randint(50,200)                       

        cv.drawContours(img, [triangle_cnt.astype(int)], 0, (r,g,b), cv.FILLED)
        cv.drawContours(img_bg, [triangle_cnt.astype(int)], 0, (r,g,b), cv.FILLED)
        
        # Bounding rect
        x,y,w,h = cv.boundingRect(triangle_cnt.astype(int))
        
        # Add line to a file content
        txt_line = txt_line + \
                   str(x) + ',' + str(y) + ',' + \
                   str(x+w-1) + ',' + str(y+h-1) + \
                   ',2,' + \
                   str(int(pt1[0])) + ',' + str(int(pt1[1])) + ',' + \
                   str(int(pt2[0])) + ',' + str(int(pt2[1])) + ',' + \
                   str(int(pt3[0])) + ',' + str(int(pt3[1])) + ' '
    
    # Stars - class 3
    stars = random.randint(0,max_stars)
    for t in range(stars):
        fine = False
        attempts = 0
        number_peaks = random.randint(4, 6)

        star_points = np.empty((number_peaks*2, 2), dtype=Point)
        distance = 360.0 / float(number_peaks)                
    
        while attempts < max_attempts:
            outer_size = random.randint(min_size_star,max_size_star)
            inner_size = int(outer_size * float(1/2))
            angle_rot = random.randint(min_angle,max_angle)
            center_x = random.randint(outer_size+1,image_w-outer_size-1)
            center_y = random.randint(outer_size+1,image_h-outer_size-1)
            
            if rotation_allowed == False:
                angle_rot = 0
            
            for p in range(number_peaks):
                angle = (p * distance) * (math.pi/180.0) + angle_rot
                angle_2 = ((p * distance) + (distance / 2)) * (math.pi/180.0) + angle_rot

                star_points[p*2,0] = center_x + (outer_size * math.sin(angle))
                star_points[p*2,1] = center_y - (outer_size * math.cos(angle))                
                star_points[p*2+1,0] = center_x + (inner_size * math.sin(angle_2))           
                star_points[p*2+1,1] = center_y - (inner_size * math.cos(angle_2))           
            
            #if rotation_allowed == True:
            #    for i in range(len(star_points)):
            #        star_points[i][0] = (star_points[i][0] - center_x) * math.cos(angle_rot) - (star_points[i][1] - center_y) * math.sin(angle_rot)
            #        star_points[i][1] = (star_points[i][1] - center_y) * math.cos(angle_rot) + (star_points[i][0] - center_x) * math.sin(angle_rot)

            #        star_points[i][0] = star_points[i][0] + center_x
            #        star_points[i][1] = star_points[i][1] + center_y

            if occlusion_allowed == True:
                break;
            
            img_shape = np.zeros((image_w,image_h,1), np.uint8)
            cv.drawContours(img_shape, [star_points.astype(int)], 0, 1, cv.FILLED)

            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            ret,img_gray = cv.threshold(img[:,:,0],0,1,cv.THRESH_BINARY)
            
            occlusion_test = cv.bitwise_and(img_shape,img_gray)
            
            white_pixels = cv.countNonZero(occlusion_test)
            
            attempts = attempts+1
            
            if (white_pixels > 0):
                continue;
            else:
                break;    
                
        if (attempts == max_attempts):
            failed = 'FAILED_'
        
        r = random.randint(50,200)
        g = random.randint(50,200)
        b = random.randint(50,200)                       

        cv.drawContours(img, [star_points.astype(int)], 0, (r,g,b), cv.FILLED)
        cv.drawContours(img_bg, [star_points.astype(int)], 0, (r,g,b), cv.FILLED)
        
        # Bounding rect
        x,y,w,h = cv.boundingRect(star_points.astype(int))
        
        # Convex hull
        hull = cv.convexHull(star_points.astype(int))
        
        # Add line to a file content
        txt_line = txt_line + \
                   str(x) + ',' + str(y) + ',' + \
                   str(x+w-1) + ',' + str(y+h-1) + \
                   ',3,'
       
        for vertex in star_points.astype(int):
            txt_line = txt_line + str(vertex[0]) + ',' + str(vertex[1]) + ','            
            
        txt_line = txt_line[:-1]
        txt_line = txt_line + ' '
        
        
    # Random polygon - class 4 
    global random_points
    randoms = random.randint(1,max_randoms)
    for t in range(randoms):
        random_points = []
        attempts      = 0
        while attempts < max_attempts:
            random_points = []
            outer_size    = random.randint(min_size_random,max_size_random)
            center_x      = random.randint(outer_size+1,image_w-outer_size-1)
            center_y      = random.randint(outer_size+1,image_h-outer_size-1)
            tot_angle     = 0

            while True:
                dist = random.randint(int(outer_size*0.5), outer_size)
                if (tot_angle==0): dist = outer_size
                
                x1   = math.cos(math.radians(tot_angle)) * dist + center_x
                y1   = math.sin(math.radians(tot_angle)) * dist + center_y
                random_points.append((x1,y1))

                tot_angle += random.randint(30,80)
                if (tot_angle > 330): break

            random_points = np.array(random_points).astype(int)        
        
        
            img_shape      = np.zeros((image_w,image_h,1), np.uint8)
            cv.drawContours(img_shape, [random_points], 0, 1, cv.FILLED)
            img_gray       = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            ret,img_gray   = cv.threshold(img[:,:,0],0,1,cv.THRESH_BINARY)
            occlusion_test = cv.bitwise_and(img_shape,img_gray)
            white_pixels   = cv.countNonZero(occlusion_test)
            attempts = attempts+1
            
            if (white_pixels > 0):
                continue;
            else:
                break;
                
        if (attempts == max_attempts): failed = 'FAILED_'        
        
        r = random.randint(0,230)
        g = random.randint(0,230)
        b = random.randint(0,230)                       

        cv.drawContours(img, [random_points], 0, (r,g,b), cv.FILLED)
        cv.drawContours(img_bg, [random_points], 0, (r,g,b), cv.FILLED)
        
        # Bounding rect
        x,y,w,h = cv.boundingRect(random_points)
        
        # Add line to a file content
        txt_line = txt_line + \
                   str(x) + ',' + str(y) + ',' + \
                   str(x+w-1) + ',' + str(y+h-1) + \
                   ',4,'
       
        for vertex in random_points:
            txt_line = txt_line + str(vertex[0]) + ',' + str(vertex[1]) + ','            
            
        txt_line = txt_line[:-1]
        txt_line = txt_line + ' '
        
    if (failed == 'FAILED_'):
        continue
        
    img_bg = np.clip(img_bg + np.random.rand() * 30.0 * np.random.random(img_bg.shape), 0, 255)
        
    file_name = failed + 'img_' + str(i) + '.png'
    file_path = image_folder + '/' + file_name    
    if (bg_folder == ''):
        cv.imwrite(file_path, img)    
    else:
        cv.imwrite(file_path, img_bg)    
    
    # Trim last space
    txt_line = txt_line[:-1]
    txt_line = image_folder + '/' + file_name + ' ' + txt_line + '\n'

    # Save to output file
    text_file = open(image_folder + '/' + txt_output, "a")
    text_file.write(txt_line)
    
    text_file.close()    
    
print('Done')